In [10]:

# Importation des librairies 

import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import time
import progressbar
import sys


from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report,roc_auc_score,precision_score,recall_score, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn_som.som import SOM
import networkx as nx


pd.set_option('display.max_columns', 100)

from GEM.gem.utils      import graph_util, plot_util
from GEM.gem.evaluation import visualize_embedding as viz
from GEM.gem.evaluation import evaluate_graph_reconstruction as gr
from GEM.gem.embedding.gf       import GraphFactorization
#from GEM.gem.embedding.sdne     import SDNE
#from argparse import ArgumentParser
#from GraphEmbedding.ge import DeepWalk
#from GraphEmbedding.ge import SDNE
from karateclub.graph_embedding import Graph2Vec
from karateclub.node_embedding.neighbourhood import HOPE
from karateclub.node_embedding.neighbourhood import DeepWalk




num_frame=200#arbitraire , a tester plus serieusement

print("import terminé a :",time.strftime("%H:%M:%S", time.localtime()))

import terminé a : 11:39:08


In [12]:
def instals():
    !pip install progressbar2
    !pip3 install PyQt5
    
    

def create_graph(X_train_ultra_simple):
    g = nx.MultiGraph()

    start = time.time()
    i=0
    while (i<len(X_train_ultra_simple)):
        a=X_train_ultra_simple["merchant"][i]
        b=X_train_ultra_simple["cc_num"][i]
        g.add_edge(a,b,weight=1,edge_id=i)
        i=i+1
    
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
    return g

def create_graph_deprecated(X_train_ultra_simple):
    g = nx.MultiGraph()

    start = time.time()
    i=0
    while (i<len(X_train_ultra_simple)):
        a=X_train_ultra_simple["merchant"][i]
        b=X_train_ultra_simple["cc_num"][i]
        g.add_edge(a,b,weight=0,edge_id=i)
        i=i+1
    i=0
    while (i<len(X_train_ultra_simple)):# on lis 2 fois mais ca coute que 4 sec
        a=X_train_ultra_simple["merchant"][i]
        b=X_train_ultra_simple["cc_num"][i]
        g[a][b]["weight"]=g[a][b]["weight"]+1 # il falais initialiser en premier
        i=i+1
    
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
    return g

def fill(g,liste):
  
    while(liste):
        a=liste.pop()
        i=0
        while(i<len(liste)):
            b=liste[i]
            g.add_edge(a,b,weight=1)
            i+=1
    
    #return g

def init_sub_graph(nb_frames):
    # divison en plusieures sous graphs 
    sous_graph=[]
    i=0
    i2=0
    sub_g=0
    while ( i<num_frame):
        sous_graph.append(nx.MultiGraph())
        i=i+1
    return sous_graph

def bipartite_dict(dict_merchants,dict_cc_num):

    dict_merchants_copy=dict_merchants.copy()
    dict_merchants_copy = dict([(value, key) for key, value in dict_merchants_copy.items()])
    dict_cc_num_copy=dict_cc_num.copy()
    dict_cc_num_copy = dict([(value, key) for key, value in dict_cc_num_copy.items()])

    for key in dict_merchants_copy.keys():
        dict_merchants_copy[key] = 0
    for key in dict_cc_num_copy.keys():
        dict_cc_num_copy[key] = 1
    return dict_merchants_copy,dict_cc_num_copy

def create_sub_graph(g,nb_frames,dict_merchants,dict_cc_num):
    sous_graph=init_sub_graph(nb_frames)
    dict_merchants_copy,dict_cc_num_copy=bipartite_dict(dict_merchants,dict_cc_num)    
    time_frame_size=len(X_train_ultra_simple) / num_frame
    start = time.time()
    connected_count=0
    i=0
    i2=0
    sub_g=0
    while (i<len(X_train_ultra_simple)):
        a=X_train_ultra_simple["merchant"][i]# rendre plus lisible
        b=X_train_ultra_simple["cc_num"][i]
        sous_graph[sub_g].add_edge(a,b,weight=1,edge_id=i,edge_rid=i2)

        i=i+1
        i2=i2+1
        if i2>= time_frame_size:
            nx.set_node_attributes(sous_graph[sub_g], dict_merchants_copy, "bipartite")
            nx.set_node_attributes(sous_graph[sub_g], dict_cc_num_copy, "bipartite")
            i2=0
            if(nx.is_connected(sous_graph[sub_g])):
                connected_count+=1
            sub_g=sub_g+1

    print("---graph split = %s seconds ---" % (time.time() - start));start = time.time()

    return sous_graph,connected_count


def create_sub_graph_deprecated(g,nb_frames,dict_merchants,dict_cc_num):
    sous_graph=init_sub_graph(nb_frames)
    dict_merchants_copy,dict_cc_num_copy=bipartite_dict(dict_merchants,dict_cc_num)    
    time_frame_size=len(X_train_ultra_simple) / num_frame
    start = time.time()
    connected_count=0
    i=0
    i2=0
    sub_g=0
    while (i<len(X_train_ultra_simple)):
        a=X_train_ultra_simple["merchant"][i]# rendre plus lisible
        b=X_train_ultra_simple["cc_num"][i]
        if(sous_graph[sub_g].has_edge(a,b)):
            sous_graph[sub_g][a][b]["weight"]=sous_graph[sub_g][a][b]["weight"]+1 
        else:
            sous_graph[sub_g].add_edge(a,b,weight=1,edge_id=i)
            #sous_graph[sub_g][a]["bipartite"]=0
            #sous_graph[sub_g][b]["bipartite"]=1
        i=i+1
        i2=i2+1
        if i2>= time_frame_size:
            nx.set_node_attributes(sous_graph[sub_g], dict_merchants_copy, "bipartite")
            nx.set_node_attributes(sous_graph[sub_g], dict_cc_num_copy, "bipartite")
            i2=0
            if(nx.is_connected(sous_graph[sub_g])):
                connected_count+=1
            sub_g=sub_g+1

    print("---graph split = %s seconds ---" % (time.time() - start));start = time.time()

    return sous_graph,connected_count


def invert_graph(gr):
    new_graph=nx.MultiGraph()
    #tous les arc d'un sommet sont connecté entre eux
    #step 1 = dans new_graph creer un sommet pour chaque arc

    for edge in gr.edges(data=True):
        #print(edge)
        node_id=edge[2]["edge_rid"]
        new_graph.add_node(node_id)

        #print(" g[edge[0]][edge[1]][edge_id] = ",g[edge[0]][edge[1]]["edge_id"])
    #pour chaque node de g relier enssemble tous les arc
    for node in gr.nodes:
        _edge_list=list(gr.edges(node,data="edge_rid"))
        node_list=[]
        i=0
        #print(_edge_list)
        for items in _edge_list:
            #print(node[0])
            node_list.append(_edge_list[i][2])
            i=i+1            
        fill(new_graph,node_list)


    return new_graph

def invert_graph_deprecated(gr):
    new_graph=nx.MultiGraph()
    #tous les arc d'un sommet sont connecté entre eux
    #step 1 = dans new_graph creer un sommet pour chaque arc

    for edge in gr.edges(data=True):
        #print(edge)
        node_id=edge[2]["edge_rid"]
        new_graph.add_node(node_id)

        #print(" g[edge[0]][edge[1]][edge_id] = ",g[edge[0]][edge[1]]["edge_id"])
    #pour chaque node de g relier enssemble tous les arc
    for node in gr.nodes:
        dico=list(gr[node])# traitement de 1 node
        #print(dico)
        node_list=[]
        for key in dico:
            
            node_id=gr[node][key][0]["edge_rid"]# to test
            node_list.append(node_id)
        fill(new_graph,node_list)


    return new_graph


def create_inverted_sub_graph(sous_graph):
    start = time.time
    inv_sous_graph=[]
    for sg in sous_graph:
        inv_sous_graph.append(invert_graph(sg))
        print(".",end="")
    return inv_sous_graph
    print("---create_inverted_sub_graph  %s seconds ---" % (time.time() - start));start = time.time



def start_time_eval():
    start = time.time()
    i=0
    while(i<1000000):
        i=i+1
    boucle_time=time.time() - start
    start = time.time()
    i=0
    while(i<1000000):
        poubelle =time.time()
        i=i+1
    print("---1 milion de time.time=  %s seconds ---" % (time.time() - start-boucle_time));start = time.time()

    


#return le nombre d'arete du graph weighted_g
def nb_edge(weighted_g):
    summ=0
    NODES = list(weighted_g.nodes)
    for node in NODES:
        summ= summ+G.degree[node]
    return summ/2

def replissement(weighted_g, nb_merc,nb_cc_num):
    nb_edges=nb_edge(weighted_g)
    nb_max_edges =(nb_merc* nb_cc_num)#graph bipati
    return  nb_edges / nb_max_edges
    
def edge_repartition(g,len_dict_merchants):
    repartition=[]
    #remplissage de repartiotion avec 0 pour eviter les bugg
    
    #for each vertex 
        # for each edge in vertex.edges
            #repartition[ edge.poid ] ++
    print ("")
    

def slow_concat(d1,d2):
    return dict(d1.items() | d2.items())

def ditc_maping_so_slow_but_why(X_train_ultra_simple,dict_merchants,dict_cc_num):
    #---dictionary maping = 4272.313026428223 seconds ---
    start = time.time()
    size =len(X_train_ultra_simple)
    i=0
    time_val=[]
    while (i<size):
        X_train_ultra_simple.iat[i,0]=dict_merchants[X_train_ultra_simple.iat[i,0]]
        X_train_ultra_simple.iat[i,1]=dict_cc_num[X_train_ultra_simple.iat[i,1]]
        time_val.append(time.time() - start);start = time.time()
        i=i+1
    return time_val


def ditc_maping(X_train_ultra_simple,dict_merchants_cc_num):
    X_train_ultra_simple["merchant"].replace(dict_merchants_cc_num, inplace=True)
    X_train_ultra_simple["cc_num"].replace(dict_merchants_cc_num, inplace=True)

def ditc_maping_2(X_train_ultra_simple,dict_merchants_cc_num):
    X_train_ultra_simple["merchant"].replace(dict_merchants_cc_num, inplace=True)
    X_train_ultra_simple["cc_num"].replace(dict_merchants_cc_num, inplace=True)

    
def create_split_dict(X_train_ultra_simple):
    start = time.time()
    dict_merchants=dict()
    dict_cc_num=dict()
    index =0
    merc_id=0
    cc_id=0
    while index < len(X_train_ultra_simple):
        if X_train_ultra_simple["merchant"][index] not in dict_merchants.keys():
            dict_merchants[X_train_ultra_simple["merchant"][index]] = merc_id
            merc_id=merc_id+1
        index=index+1

    print("---remplissage dict_merchants  %s seconds ---" % (time.time() - start));start = time.time()
    index=0
    while index < len(X_train_ultra_simple):
        if X_train_ultra_simple["cc_num"][index] not in dict_cc_num.keys():
            dict_cc_num[X_train_ultra_simple["cc_num"][index]] = merc_id
            merc_id=merc_id+1
        index=index+1

    print("---remplissage dict_cc_num %s seconds ---" % (time.time() - start));start = time.time()
    return dict_merchants,dict_cc_num

def create_dict(X_train_ultra_simple):
    dict_merchants,dict_cc_num=create_split_dict(X_train_ultra_simple)
    
    return slow_concat(dict_merchants,dict_cc_num)
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
def create_dictionary_hell():# missing double values
    start = time.time()
    dictionary_hell=[]
    nodes=[]
    nb_sg=len(sous_graph)
    print(len(sous_graph))
    sg_index=0
    merc_id=0
    while sg_index < nb_sg:
        i=0
        merc_id=0
        nodes= list(sous_graph[sg_index].nodes)
        dictionary_hell.append(dict())
        while( i<len(nodes)):
            if nodes[i] not in dictionary_hell[sg_index].keys():
                dictionary_hell[sg_index][nodes[i]] = merc_id
                merc_id=merc_id+1
                
            i=i+1

        sg_index=sg_index+1
    print("--- dictionary__hell %s seconds ---" % (time.time() - start));start = time.time()
    return dictionary_hell

def relabel_graph(r_sous_graph,dictionary_hell,sous_graph):
    print ("relabel_",end="")
    start = time.time()
    i=0  
    while i<len(sous_graph):
        r_sous_graph.append(nx.relabel_nodes(sous_graph[i], dictionary_hell[i],copy=True))
        i=i+1
        print(".",end="")
    print("--- relabel_graph %s seconds ---" % (time.time() - start))

def unrelabel_graph(unr_sous_graph,dictionary_hell,sous_graph):
    start = time.time()
    i=0
    while i<len(dictionary_hell):
        if i==len(sous_graph):
            (print("errno=",i,end=','))
        inv_map = {v: k for k, v in dictionary_hell[i].items()}
        unr_sous_graph.append(nx.relabel_nodes(sous_graph[i], inv_map,copy=True))
        i=i+1
    print("--- revert labelling %s seconds ---" % (time.time() - start))


def print_info_diverses(X_train_ultra_simple,dico):
    print(X_train_ultra_simple["merchant"][0])
    print(X_train_ultra_simple["cc_num"][0])
    print(X_train_ultra_simple.columns)
    print(X_train_ultra_simple.loc[0]["merchant"])

    print (len(dico) , "humans in the system ")#1676
    print(X_train_ultra_simple["cc_num"][0])
    print (dict_merchants[ "fraud_Rippin, Kub and Mann"], type(dico[ "fraud_Rippin, Kub and Mann"]))
    

#laplacian similarity 1/2
def select_k(spectrum, minimum_energy = 0.9):#
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

#laplacian similarity 2/2
def laplacian_similarity(graph1,graph2):
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)
    print("k selected =",k)
    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    return similarity

def string_edit_dist():
    print("https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Levenshtein.html")
def edit_dist_nx(g1,g2):
    
    for v in nx.optimize_graph_edit_distance(g1, g2):
        minv = v
    return minv


def graph_degree(g):
    degrees = [val for (node, val) in g.degree()]
    maxd=max(degrees)
    avg=sum(degrees)/len(degrees)
    print("max degree =",maxd," mean = ",avg)
    return maxd,avg
def print_graph_info(connected_count):
    
    print(connected_count,"connected graphs")
    print(len(sous_graph[0].edges)," transactions ")
    print("sous_graph[0][0] , type = ",type(sous_graph[0][0]),"\n")
    print("sous_graph[0].nodes , type = ",type(sous_graph[0].nodes),"\n")
    print(list(sous_graph[0].nodes))
    print("------------")
    sub_g=0;node=0
    dico=dict(sous_graph[sub_g][node])
    for key in dico:
         print(sous_graph[0][0][key],"key = ",key)
    print(dict(sous_graph[0][0]))
    print("------------")
    print( type(sous_graph[0][0][693]))
    print(sous_graph[0][0][693])
    print("---------")
    print((sous_graph[0].edges))
    
def draw_1(g):
    start = time.time()
    #subax1 = plt.subplot()
    nx.draw(sous_graph[0], with_labels=False, node_size= 1)
    plt.savefig("draw_1.png")
    plt.show()
    print("---draw  %s seconds ---" % (time.time() - start));start = time.time
def draw_2(g):
    start = time.time()
    #subax2 = plt.subplot()
    options = {
        'node_size': 100,
        'width': 3,
    }
    nx.draw_spectral(g, **options)#approximation of the ratio cut
    plt.savefig("draw_2.png")
    plt.show()
    print("---draw  %s seconds ---" % (time.time() - start));start = time.time
def draw_3(g):
    start = time.time()
    #subax3 = plt.subplot()

    nx.draw_shell(g, with_labels=False,node_size= 1)# font_weight='bold')
    plt.savefig("draw_3.png")
    plt.show()
    print("---draw  %s seconds ---" % (time.time() - start));start = time.time
    
def draw_4(g,numb_merchant):
    total=len(g.nodes)#les valeures ascossié ne sont pas les bonnes mais c'est 
    X = list(range(0,numb_merchant ))# juste pour la position geographique 
    Y= list(range(numb_merchant,total ))
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
    nx.draw(g, pos=pos,node_size= 1)
    plt.savefig("draw_4.png")
    plt.show()

def drawing(g):
    %matplotlib inline
    draw_1(g)
    draw_2(g)
    draw_3(g)
    draw_4(g,len(dict_merchants.keys()))
    
    plt.show()
    
def complexity_calculations():
    val=0
    train_size=len(Y_train)
    test_size=len(Y_test)
    
    #l'idee premiere
    print("-----------------")
    print("tester la similarité de 1 graph modifié avec tt les autres ")
    print("pour 0.5 sec par calcul et ",num_frame," frames" )
    print( num_frame*0.5," sec")
    print("-----------------")
    print("modifier le graph et recomencer , pour chaques valeures dans train")
    print("pour completement calculer  les similarité de 1 transaction")
    print( num_frame*0.5*train_size," sec")
    print("donc ",int(num_frame*0.5*train_size/(3600*24))," jours")
    print("-----------------")
    
    big_number = num_frame*0.5*train_size*test_size
    big_number_year=int(big_number/31540000)
    print("pour un total de ",big_number," sec")
    print("donc ",big_number_year," ans")
    print("-----------------")
    print("en reduisant la precision au minimum")
    print("chaque transaction n'aura que 1 calcul de similarité")
    print("precision max 50% , doubler le temps de calcul double la precision")
    big_number=0.5*test_size
    print("pour un total de ",big_number," sec")
    print("donc ",int(big_number/3600)," heures")
    
def print_version():
    print ("python ",sys.version)


def hope_on_inv_sg(inv_sg):

    start = time.time()
    # hope attributed example
    model = HOPE(dimensions=58)
    liste=[]
    i=0
    process_bar = progressbar.ProgressBar().start(max_value=num_frame)
    while (i<num_frame):
        # train the model and generate embeddings
        process_bar.update(i)
        model.fit(inv_sg[i])
        liste.append(model.get_embedding())
        i+=1
    print("---hope embedding = %s seconds ---" % (time.time() - start));start = time.time()
    return liste  

def deepwalk_on_inv_sg(inv_sg):

    start = time.time()
    # hope attributed example
    model = DeepWalk(dimensions=58)
    liste=[]
    i=0
    while (i<num_frame):
        # train the model and generate embeddings
        model.fit(inv_sg[i])
        liste.append(model.get_embedding())
        i+=1
    print("---deepwalk embedding = %s seconds ---" % (time.time() - start));start = time.time()
    return liste      


print("fonctions declaré a :",time.strftime("%H:%M:%S", time.localtime()))

fonctions declaré a : 11:40:52


In [3]:
##download data

program_start = time.time()

import os
data_file= os.path.abspath('../../data')
full_path=data_file+'\\'+'fraudTrain.csv'
train_df=pd.read_csv(full_path)
full_path=data_file+'\\'+'fraudTest.csv'
test_df=pd.read_csv(full_path)

cols = train_df.columns.tolist()
cols = [c for c in cols if c not in ["is_fraud"]]
target = "is_fraud"
print(cols)

#Definition des nouvelles variables X_train and Y_train
X_train = train_df[cols]
Y_train = train_df[target]

#Definition des nouvelles variables X_test and Y_test
X_test = test_df[cols]
Y_test = test_df[target]

features = [ 'merchant', 'cc_num']
X_train = X_train[features]
X_test = X_test[features]

X_train_ultra_simple = X_train.copy()
X_test_ultra_simple = X_test.copy()


['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long']


In [4]:

#remplissage des dictionaires

dict_merchants,dict_cc_num=create_split_dict(X_train_ultra_simple)
dictionary=slow_concat(dict_merchants,dict_cc_num)
#dictionary=create_dict(X_train_ultra_simple)

# associer a chaque marchant son numero dans le dictionaire
#pour la lisibilité , et l'affichage
start = time.time()
ditc_maping(X_train_ultra_simple,dictionary)

print("---dictionary maping = %s seconds ---" % (time.time() - start));start = time.time()


#pip install tk

    ##creation du graph
    

g = create_graph(X_train_ultra_simple)#40 sec

#!jupyter notebook --generate-config
# divison en plusieures sous graphs #20 sec

sous_graph,connected_count=create_sub_graph(g,num_frame,dict_merchants,dict_cc_num)
#dictionary_hell=create_dictionary_hell()

#pour afficher , attention au cascades
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


print(connected_count," connected graphs")
i=3
#while(i<num_frame):# doesnt work or my graph are the same
while(i<3):
    start = time.time()

    print ("-------------------------------------")
    #laplacian marche
    val = laplacian_similarity(sous_graph[0],sous_graph[i])
    #val1=nx.graph_edit_distance(sous_graph[0],sous_graph[i],timeout=60,upper_bound=1e10)
    #val2=nx.graph_edit_distance(sous_graph[0],sous_graph[i],timeout=120,upper_bound=1e10)
    #val3=nx.graph_edit_distance(sous_graph[0],sous_graph[i],timeout=180,upper_bound=1e10)
    #print ("comparing ",hex(id(sous_graph[0]))," and ",hex(id(sous_graph[i])))
    #print (val1," ",val2," ",val3) 
    print(sous_graph[0])
    print(sous_graph[i]) 
    print(" similarity = ",val)
    print("---laplacian similarity = %s seconds ---" %         f"{(time.time() - start):.4}"    );start = time.time()

    print ("-------------------------------------")
    i=i+1
''' 
plan,strategie , objectif
changement de plan
probleme , le poid des arc devien quoi ? 
comment ajouter amt apres coup , il suffit de l'ajouter sur les donnée
avan le SOM
                                            probably better
j'inverse le graph                |  je le coupe en n sous_graphs
je le coupe en n sous_graphs      |  j'inverse les sous_graphs


je compresse les sous graphs 
je SOM sur les cous graphs

complexité= O(n*tps_compression + SOM) // SOM = 6min = negligeable
'''    


print("---depuis le debut  %s seconds ---" % (time.time() - program_start));start = time.time
#!git clone https://github.com/shenweichen/GraphEmbedding.git

def int_to_str(G):
    # convert nodes from int to str format
    keys = np.arange(0,int(len(dictionary.keys())))
    values = [str(i) for i in keys]
    dic = dict(zip(keys, values))
    H = nx.relabel_nodes(G, dic)

def my_fun():
    print('How many cats do you have?\n')
    numCats = input()
    try:
        if int(numCats) > 3:
            print('That is a lot of cats.')
        else:
            print('That is not that many cats.')
    except ValueError:
        print("Value error")  
def modif_graph_testing(graph):
    graph[1][2]["weight"]=2
    
def invert_node_testing():
    graph=nx.MultiGraph()
    graph.add_nodes_from([0,1,2,3,4])
    graph.add_edge(0,1,weight=0,edge_id="a")
    graph.add_edge(1,2,weight=0,edge_id="b")
    graph.add_edge(2,3,weight=0,edge_id="c")
    graph.add_edge(3,4,weight=0,edge_id="d")
    graph.add_edge(4,2,weight=0,edge_id="e")
    print(graph.nodes)
    print(graph.edges)
    print("avan 1--2--3--4")
    #new_graph=inver t_graph(graph)
    print("apres 1--2--3--4")
    print(new_graph.nodes)
    print(new_graph.edges)
    
    
embedings=list()
#all_embeding(embedings,sous_graph)
#all_embeding(embedings,sous_graph)
print("try")

if(False):
    liste=[]
    i=0
    num_frame=len(sous_graph)
    G = sous_graph[i]
    liste.append(encapsulation(G))
    i+=1

    G = sous_graph[i]
    liste.append(encapsulation(G))
    i+=1

#crash_free()
#!pip3 install PyQt5
# affichage
print(sous_graph[0])
#drawing(sous_graph[0])

print(len(sous_graph[0].nodes))
print(len(g.nodes))


#370 MiB = 300 000 Mo
# avec g/200 ca compile , tres .... lentement , mais ca compile
too_long=True
if(not too_long):
    val = edit_dist_nx(sous_graph[0],sous_graph[1])


affichage=False
if(affichage):
    complexity_calculations()
    drawing(g)
    print_graph_info(connected_count)


### partie karateclub



---remplissage dict_merchants  9.19101858139038 seconds ---
---remplissage dict_cc_num 13.082401275634766 seconds ---
---dictionary maping = 63.52919626235962 seconds ---
---graph construction = 25.92681312561035 seconds ---
---graph split = 29.92465877532959 seconds ---
189  connected graphs
---depuis le debut  150.36047887802124 seconds ---
try
MultiGraph with 1583 nodes and 6484 edges
1583
1676


In [5]:
### partie karateclub
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################

In [6]:
#graph construction



start = time.time()
inv_sg = create_inverted_sub_graph(sous_graph)

#r_sous_graph=[]
#relabel_graph(r_sous_graph,dictionary_hell,inv_sg)
#unr_sous_graph=[]
#unrelabel_graph(unr_sous_graph,dictionary_hell,r_sous_graph)

print("--- graph construction = %s seconds ---" % int((time.time() - start)));start = time.time()


........................................................................................................................................................................................................--- graph construction = 71.24076771736145 seconds ---


In [7]:
#inv_sg = create_inverted_sub_graph(sous_graph)
#for node in inv_sg[30].nodes(data=True):
    #print(node)
def is_it_missing_a_number(whatever):
    missing=False
    if isinstance(whatever,dict):
    
        l=sorted(whatever.keys())
        i=0
        while i<len(l):
            if l[i]!=i:
                missing=True
            i=i+1
        i=i-1
        #print("len(dict)= ",len(l)," , lastval = ",l[i],end=" ")
        if(missing):
            print(" missing a number",end=" ")
            return True
    if isinstance(whatever,list):
        l=sorted(whatever)
        i=0
        while i<len(l):
            if l[i]!=i:
                missing=True
            i=i+1
        i=i-1
        #print("len(list)= ",len(l)," , lastval = ",l[i],end=" ")
        if(missing):
            print(" missing a number",end=" ")
            return True
    #print("\n")
    return False
    
def is_inv_working():
    print (len(inv_sg[30].nodes))
    l1=list(inv_sg[30].nodes)
    l2=list(unr_sous_graph[30].nodes)
    i=0
    while(i<len( inv_sg[30])):
        if(l1[i] != l2[i]):
            print("not_the_same")
        i=i+1


    i=0# il y a au moins les memes values
    for k in inv_sg[30] :
        #print(",",end="");i=i+1
        if k not in(unr_sous_graph[30] ):
            print("not_the_same")
    print(i)

In [8]:
#print(sorted(list(r_sous_graph[0].nodes)))
#print(sorted((dictionary_hell[0]).keys()))
#is_it_missing_a_number(dictionary_hell[1])
i=0
while(i<len(inv_sg)):
    nodes = inv_sg[i].nodes(data=False)
    is_it_missing_a_number(list(nodes))
    i=i+1

In [13]:
#---hope embedding = 354.7910692691803 seconds ---
hope_emb = hope_on_inv_sg(inv_sg)# doc reading needed , for node order


    
    
    
    
    
    
# Graph2Vec attributed example
if(False):
    small_sous_graph=[r_sous_graph[0],r_sous_graph[1]]
    
    model = Graph2Vec(attributed=False)
    model.fit(small_sous_graph)
    ecmb=model.get_embedding()

    print(type(range(r_sous_graph[0].number_of_nodes())))

    print("---embedding = %s seconds ---" % (time.time() - start));start = time.time()


 99% (199 of 200) |##################### | Elapsed Time: 0:05:04 ETA:   0:00:01

---hope embedding = 305.74938702583313 seconds ---


In [ ]:
#deepwalk_emb=deepwalk_on_inv_sg(inv_sg)

In [19]:
graph=nx.MultiGraph()
if(True):
    graph.add_edge(0,1,weight=1,edge_id="a")
    graph.add_edge(0,2,weight=1,edge_id="b")
    graph.add_edge(0,3,weight=2,edge_id="c")
    graph.add_edge(0,4,weight=1,edge_id="d")
    graph.add_edge(3,4,weight=1,edge_id="e")
    graph.add_edge(4,3,weight=1,edge_id="e")
print(graph.edges(data=True))
l=graph.edges(3,data="edge_id");i=0
print(list(l)[1])


for edge in graph.edges(data=True):
    count_down=edge[2]["weight"]
    while(count_down>0):
        print(count_down)
        count_down=count_down-1
if(True):
    model = HOPE(dimensions=6)
    model.fit(graph)
    emm=model.get_embedding()
    print(emm)

[(0, 1, {'weight': 1, 'edge_id': 'a'}), (0, 2, {'weight': 1, 'edge_id': 'b'}), (0, 3, {'weight': 2, 'edge_id': 'c'}), (0, 4, {'weight': 1, 'edge_id': 'd'}), (3, 4, {'weight': 1, 'edge_id': 'e'}), (3, 4, {'weight': 1, 'edge_id': 'e'})]
(3, 4, 'e')
1
1
2
1
1
1
1
[[ 0.51002663 -0.9308012  -2.609807    0.51002866 -0.9308012  -2.6098063 ]
 [ 0.8970183   0.35892105 -0.73470134  0.897021    0.35892007 -0.7347016 ]
 [ 0.89701784  0.358919   -0.73470145  0.8970208   0.35891926 -0.7347016 ]
 [-0.43517843  1.0882282  -2.7570646  -0.43518016  1.0882277  -2.7570655 ]
 [-0.6337237  -0.48032188 -2.2870173  -0.63372576 -0.48032126 -2.2870164 ]]


In [14]:
_max=0
for gr in inv_sg:
    current_max,mean =graph_degree(gr)
    if current_max>_max:
        _max=current_max
print ("max degree= ",_max)

max degree = 43  mean =  21.30197409006786
max degree = 49  mean =  21.444478716841456
max degree = 51  mean =  21.63479333744602
max degree = 47  mean =  21.35502776064158
max degree = 48  mean =  21.64497223935842
max degree = 52  mean =  21.529302899444787
max degree = 48  mean =  21.410549043800124
max degree = 50  mean =  21.451573103022824
max degree = 46  mean =  21.439543491671806
max degree = 51  mean =  21.539481801357187
max degree = 53  mean =  21.214682294879704
max degree = 47  mean =  21.45404071560765
max degree = 48  mean =  21.50493522516965
max degree = 46  mean =  21.39296730413325
max degree = 49  mean =  21.433991363355954
max degree = 46  mean =  21.39173349784084
max degree = 48  mean =  21.43769278223319
max degree = 49  mean =  21.261875385564466
max degree = 48  mean =  21.210672424429365
max degree = 49  mean =  21.276064157927205
max degree = 58  mean =  21.445712523133867
max degree = 47  mean =  21.56323257248612
max degree = 57  mean =  21.88741517581739

max degree = 48  mean =  21.314003701418876
max degree = 45  mean =  21.45003084515731
max degree = 48  mean =  22.05305367057372
max degree = 54  mean =  21.55737199259716
max degree = 54  mean =  21.561998766193707
max degree = 45  mean =  21.362739049969154
max degree = 54  mean =  21.395816952350998
max degree=  58
